# Определение текущего уровня потребительской лояльности (NPS) телекоммуникационной компании

Заказчик этого исследования — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. Перед компанией стоит задача определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России. 

**Цель исследования** - исследование полученных результатов NPS-опросов

**Ход исследования**
- подключение к базе данных
- выгрузка данных
- создание дашборда в Tableau
- оформление результатов в виде презентации

In [1]:
#загружаем библиотеки
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
#считываем csv-файл
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

In [3]:
if os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

In [4]:
#создаём подключение к базе
if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

In [5]:
query = """
SELECT u.user_id as user_id,
        u.lt_day as lt_day,
        (CASE
        WHEN u.lt_day < 366 THEN "new"
        ELSE "old"
        END) as is_new,
        u.age as age,
        (CASE
        WHEN u.gender_segment = 1 THEN "man"
        ELSE "women"
        END) as gender_segment,
        u.os_name as os_name, 
        u.cpe_type_name as cpe_type_name,
        l.country as country,
        l.city as city,
        aseg.title as age_segment,
        tr.title as traffic_segment,
        lt.title as lifetime_segment, 
        u.nps_score as nps_score,
        (CASE 
        WHEN u.nps_score BETWEEN 9 AND 10 THEN "promoters"
        WHEN u.nps_score BETWEEN 7 AND 8 THEN "neutral"
        ELSE "other"
        END) as nps_group
FROM user AS u
LEFT JOIN location AS l ON u.location_id=l.location_id
LEFT JOIN age_segment AS aseg ON u.age_gr_id=aseg.age_gr_id
LEFT JOIN traffic_segment AS tr ON u.tr_gr_id=tr.tr_gr_id
LEFT JOIN lifetime_segment AS lt ON u.lt_gr_id=lt.lt_gr_id
"""

In [6]:
queried_df = pd.read_sql(query, engine)
queried_df.head(10)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,old,45.0,man,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,04 1-5,08 36+,10,promoters
1,A001WF,2344,old,53.0,women,ANDROID,SMARTPHONE,Россия,Киров,05 45-54,04 1-5,08 36+,10,promoters
2,A003Q7,467,old,57.0,women,ANDROID,SMARTPHONE,Россия,Москва,06 55-64,08 20-25,06 13-24,10,promoters
3,A004TB,4190,old,44.0,man,IOS,SMARTPHONE,Россия,РостовнаДону,04 35-44,03 0.1-1,08 36+,10,promoters
4,A004XT,1163,old,24.0,women,ANDROID,SMARTPHONE,Россия,Рязань,02 16-24,05 5-10,08 36+,10,promoters
5,A005O0,5501,old,42.0,man,ANDROID,SMARTPHONE,Россия,Омск,04 35-44,05 5-10,08 36+,6,other
6,A0061R,1236,old,45.0,women,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,06 10-15,08 36+,10,promoters
7,A009KS,313,new,35.0,women,ANDROID,SMARTPHONE,Россия,Москва,04 35-44,13 45-50,05 7-12,10,promoters
8,A00AES,3238,old,36.0,man,ANDROID,SMARTPHONE,Россия,СанктПетербург,04 35-44,04 1-5,08 36+,10,promoters
9,A00F70,4479,old,54.0,man,ANDROID,SMARTPHONE,Россия,Волгоград,05 45-54,07 15-20,08 36+,9,promoters


In [7]:
queried_df.to_csv('telecomm_csi_tableau.csv', index=False)

Ссылка на презентацию:

https://public.tableau.com/app/profile/petr.afonin/viz/NPS_17187247428340/NPS_8